In [136]:
from MOF_build.functions.read import read
from MOF_build.functions.normv import normalize_vector
from MOF_build.functions.rotate import rotate
import pandas as pd

In [137]:
#methyl=read.pdb('/Users/chenxili/GitHub/MOF_build/UIO66_OLD/methyl.pdb')
terms=read.pdb('/Users/chenxili/GitHub/MOF_build/DEMO/mof_build_demo/MOF_build/lib/UIO66_lib_cut.lib')

In [138]:
def get_new_cut(cut):
    C1 = cut.loc[2, ["x", "y", "z"]].to_numpy()
    CC = normalize_vector(C1) 
    methyl=read.pdb('/Users/chenxili/GitHub/MOF_build/UIO66_OLD/methyl.pdb')
    methylH1=methyl.loc[1, ["x", "y", "z"]].to_numpy()
    methylH2=methyl.loc[2, ["x", "y", "z"]].to_numpy()
    methylH3=methyl.loc[3, ["x", "y", "z"]].to_numpy()
    methylCH = (methylH1+methylH2+methylH3)/3 #methyl C is 0 0 0 
    p1 = methylCH
    p2 = C1
    methyl_arr = methyl.loc[:, ["x", "y", "z"]].to_numpy()

    q=rotate.calculate_q_rotation_with_vectors(p1,p2)

    new_methyl=rotate.get_rotated_array(methyl_arr,q)+1.5*CC+C1
    df_carbo = cut.iloc[0:3,:]

    methyl.loc[:, ["x", "y", "z"]] = new_methyl
    df_newcut = pd.concat([df_carbo,methyl],ignore_index=True,join='outer')
    return df_newcut

In [139]:

for i in range(1):
    start = 3*i
    cut=terms[start:start+3].reset_index(drop=True)
    init=get_new_cut(cut)
df_cut = init
for i in range(1,12):
    start = 3*i
    cut=terms[start:start+3].reset_index(drop=True)
    df_cut=pd.concat([df_cut,get_new_cut(cut)],join='outer',ignore_index=True)


In [140]:
from MOF_build.functions.output import output
df_left = df_cut[["Atom_label", "Residue", "Res_number", "Note"]]
df_right = df_cut[["x","y","z"]]
df = pd.concat([df_left, df_right], axis=1, join="outer")
df.to_csv("newlibcut.txt", header=None, sep="\t", index=False)
output('newlibcut',True,True,True)